In [1]:
### Import packages ###
import itertools
import pandas as pd

# Input

In [2]:
# Input Data Set #
Data ="COMPAS"
JobNameAbbrev = "CP"

# Input directory #
dir_path = "/Users/simondn/Documents/RashomonActiveLearning/Code/raw"

# Input Parameters #
ParameterDictionary = {"Data":[Data],
                     #   "Seed":list(range(0,100)),
                       "Seed":list([]),                       
                       "TestProportion":[0.2],
                       "CandidateProportion":[0.8],
                       "SelectorType":["TreeEnsembleQBCFunction"],
                       "ModelType":["TreeFarmsFunction"],
                       "UniqueErrorsInput": [0,1],
                       "n_estimators": [100], 
                       "regularization": [0.01],
                       "RashomonThresholdType": ["Adder"],                                         # ["Adder", "Multiplier"]
                       "RashomonThreshold": [0.055],
                       "Type": ["Classification"],
                       "Partition": ["short"],                                                        # [short, medium, long, largemem, compute, cpu-g2-mem2x]
                       "Time": ["00:59:00"],                                                            # [00:59:00, 11:59:00, 6-23:59:00]
                       "Memory": ["100M"]}                                                                # [100M, 30000M, 100000M]

# Create Parameter Vector #
ParameterVector = pd.DataFrame.from_records(itertools.product(*ParameterDictionary.values()), columns=ParameterDictionary.keys())

# Include/exclude Passive Learning

In [3]:
### Include Random Forest ###
RandomForestParameterDictionary = {"Data":[Data],
                       "Seed":list([0, 1, 2, 3, 5, 6, 7, 9, 10, 11, 12, 13, 14, 17, 18, 21, 23, 24, 26, 27, 28, 30, 31, 34, 36, 37, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 54, 56, 59, 61, 63, 64, 65, 66, 67, 68, 70, 71, 72]),
                       "TestProportion":[0.2],
                       "CandidateProportion":[0.8],
                       "SelectorType":["PassiveLearning"],
                       "ModelType":["RandomForestClassificationFunction"],
                       "UniqueErrorsInput": [1],
                       "n_estimators": [100], 
                       "regularization": [0.01],
                       "RashomonThresholdType": ["Adder"],                                                    # ["Adder", "Multiplier"]
                       "RashomonThreshold": [0],
                       "Type": ["Classification"],
                       "Partition": ["short"],                                                        # [short, medium, long, largemem, or compute]
                       "Time": ["00:59:00"],                                                            # [00:59:00, 11:59:00, 6-23:59:00]
                       "Memory": [1000]}                                                                # [1000, 30000, 100000]

RandomForestParameterVector = pd.DataFrame.from_records(itertools.product(*RandomForestParameterDictionary.values()), columns=RandomForestParameterDictionary.keys())

# NOTE: Comment out chunk to not include random forest simulations. ###
ParameterVector = pd.concat([ParameterVector, RandomForestParameterVector]) # NOTE: Comment out to not include random forest baseline
ParameterVector = ParameterVector.sort_values("Seed")
ParameterVector.index = range(0, ParameterVector.shape[0])

/var/folders/10/2lfzfs1j0j933_mjkrskp5kh0000gq/T/ipykernel_95324/1565297613.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ParameterVector = pd.concat([ParameterVector, RandomForestParameterVector]) # NOTE: Comment out to not include random forest baseline


# Include/exclude Random Forest Simulations

In [4]:
### Include Random Forest ###
RandomForestParameterDictionary = {"Data":[Data],
                       "Seed":list([0, 1, 2, 3, 5, 6, 7, 9, 10, 11, 12, 13, 14, 17, 18, 21, 23, 24, 26, 27, 28, 30, 31, 34, 36, 37, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 54, 56, 59, 61, 63, 64, 65, 66, 67, 68, 70, 71, 72]),
                       "TestProportion":[0.2],
                       "CandidateProportion":[0.8],
                       "SelectorType":["TreeEnsembleQBCFunction"],
                       "ModelType":["RandomForestClassificationFunction"],
                       "UniqueErrorsInput": [0],
                       "n_estimators": [100], 
                       "regularization": [0.01],
                       "RashomonThresholdType": ["Adder"],                                                    # ["Adder", "Multiplier"]
                       "RashomonThreshold": [0],
                       "Type": ["Classification"],
                       "Partition": ["short"],                                                        # [short, medium, long, largemem, or compute]
                       "Time": ["00:59:00"],                                                            # [00:59:00, 11:59:00, 6-23:59:00]
                       "Memory": [1000]}                                                                # [1000, 30000, 100000]

RandomForestParameterVector = pd.DataFrame.from_records(itertools.product(*RandomForestParameterDictionary.values()), columns=RandomForestParameterDictionary.keys())

# NOTE: Comment out chunk to not include random forest simulations. ###
ParameterVector = pd.concat([ParameterVector, RandomForestParameterVector]) # NOTE: Comment out to not include random forest baseline
ParameterVector = ParameterVector.sort_values("Seed")
ParameterVector.index = range(0, ParameterVector.shape[0])

# Job and Output Name

In [5]:
### Remove Dictionary ###
del ParameterDictionary

# Generate JobName #
ParameterVector["JobName"] = (
    ParameterVector["Seed"].astype(str) +
    JobNameAbbrev + 
    "_MT" + ParameterVector["ModelType"].astype(str) +
    "_UEI" + ParameterVector["UniqueErrorsInput"].astype(str) +
    "_" + ParameterVector["RashomonThresholdType"].astype(str) + 
    ParameterVector["RashomonThreshold"].astype(str))

# Replace Job Name #
ParameterVector["JobName"] = (
    ParameterVector["JobName"]
    .str.replace(r"_MTTreeFarmsFunction_UEI0_", "_D", regex=True)
    .str.replace(r"_MTTreeFarmsFunction_UEI1_", "_U", regex=True)
    .str.replace(r"Adder", "A", regex=True)
    .str.replace(r"Multiplier", "M", regex=True)
    .str.replace(r"_MTRandomForestClassificationFunction_UEI0_", "_RF", regex=True)
    .str.replace(r"_MTRandomForestClassificationFunction_UEI1_", "_PL", regex=True)
    )

# Output Name #
ParameterVector["Output"] = ParameterVector["Data"].astype(str) + "/" + ParameterVector["ModelType"].astype(str) + "/Raw/" + ParameterVector["JobName"] + ".pkl"
ParameterVector["Output"] = ParameterVector["Output"].str.replace("Function", "", regex=False)

---

# Only run these ones

In [6]:
# # ### Simulations that failed ###
# FilterText = ["73B7_UA0.02", "68B7_UA0.02", "52B7_UA0.02", "80B7_UA0.02"]

# ### Rerun these simulations ###
# ParameterVector = ParameterVector[ParameterVector["JobName"].isin(FilterText)]
# ParameterVector.index = range(0,len(ParameterVector))
# ParameterVector

---

In [7]:
ParameterVector

,Data,Seed,TestProportion,CandidateProportion,SelectorType,ModelType,UniqueErrorsInput,n_estimators,regularization,RashomonThresholdType,RashomonThreshold,Type,Partition,Time,Memory,JobName,Output
0,COMPAS,0,0.2,0.8,PassiveLearning,RandomForestClassificationFunction,1,100,0.01,Adder,0,Classification,short,00:59:00,1000,0CP_PLA0,COMPAS/RandomForestClassification/Raw/0CP_PLA0...
1,COMPAS,0,0.2,0.8,TreeEnsembleQBCFunction,RandomForestClassificationFunction,0,100,0.01,Adder,0,Classification,short,00:59:00,1000,0CP_RFA0,COMPAS/RandomForestClassification/Raw/0CP_RFA0...
2,COMPAS,1,0.2,0.8,PassiveLearning,RandomForestClassificationFunction,1,100,0.01,Adder,0,Classification,short,00:59:00,1000,1CP_PLA0,COMPAS/RandomForestClassification/Raw/1CP_PLA0...
3,COMPAS,1,0.2,0.8,TreeEnsembleQBCFunction,RandomForestClassificationFunction,0,100,0.01,Adder,0,Classification,short,00:59:00,1000,1CP_RFA0,COMPAS/RandomForestClassification/Raw/1CP_RFA0...
4,COMPAS,2,0.2,0.8,PassiveLearning,RandomForestClassificationFunction,1,100,0.01,Adder,0,Classification,short,00:59:00,1000,2CP_PLA0,COMPAS/RandomForestClassification/Raw/2CP_PLA0...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,COMPAS,70,0.2,0.8,PassiveLearning,RandomForestClassificationFunction,1,100,0.01,Adder,0,Classification,short,00:59:00,1000,70CP_PLA0,COMPAS/RandomForestClassification/Raw/70CP_PLA...
96,COMPAS,71,0.2,0.8,TreeEnsembleQBCFunction,RandomForestClassificationFunction,0,100,0.01,Adder,0,Classification,short,00:59:00,1000,71CP_RFA0,COMPAS/RandomForestClassification/Raw/71CP_RFA...
97,COMPAS,71,0.2,0.8,PassiveLearning,RandomForestClassificationFunction,1,100,0.01,Adder,0,Classification,short,00:59:00,1000,71CP_PLA0,COMPAS/RandomForestClassification/Raw/71CP_PLA...
98,COMPAS,72,0.2,0.8,PassiveLearning,RandomForestClassificationFunction,1,100,0.01,Adder,0,Classification,short,00:59:00,1000,72CP_PLA0,COMPAS/RandomForestClassification/Raw/72CP_PLA...


# Save Parameter Vector

In [8]:
### Resave ###
output_path = "/Users/simondn/Documents/RashomonActiveLearning/Data/ParameterVectors/ParameterVector" + str(Data) + ".csv"
ParameterVector.to_csv(output_path, index=False)